In [ ]:
# !pip install torchmetrics
# !pip install git+https://github.com/phasepack/fasta-python 
# !pip install scikit-image
# !pip install pytorch-msssim
# !pip install wandb
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from Models import SIREN # SCAN
import matplotlib.pyplot as plt
from PIL import Image,ImageChops
from torchvision.transforms import Resize

import time
import torch
import numpy as np
from Formation import simulation, registration, evaluate
import random
import time

manualSeed = 402
# manualSeed = random.randint(1, 10000)  # use if you want new results
print("Random Seed: ", manualSeed)
torch.manual_seed(manualSeed)

# We provide three pairs of amplitude and phase in function simulation. 
# You can input your own simulated set by amp_name, phase_name = 'image path', 'image path'
# amp_name: './image/triangle.jpg',   'cameraman', 'identity', 'cameraman'
# phase_name: './image/trianglepha.jpg','identity',  'cameraman', 'brick'
amp_name= 'cameraman' 
phase_name= 'brick'
ampG = True # True False
phaG = True

# If part of the image: image_size = 256, probe_size = 50, pad = 0
# If full image: image_size = 50, probe_size = 50, pad = 103. Rescale to 50, than pad ((103,103),(103,103))
image_size = 50
probe_size = 50
pad = 103

# Simulate the CDI patterns
# y: the CDI patterns, sqrt(intensity)
# probe: known probe, for CDI is an identity matrix
# amplitude: groud truth amplitude
# phase: groud truth phase
y, probe, amplitude, phase, _ = simulation(imagesize=image_size, probesize = probe_size, pad=pad, amp_name = amp_name, phase_name = phase_name, amp_needgrayscale=ampG, pha_needgrayscale=phaG)



Random Seed:  402


In [ ]:
# SCAN
t0 = time.time()
rec_amp,rec_phase = SIREN(y, probe, probe_size, image_size, amplitude, phase, amp_name, phase_name, epoches = 5000, registration=False)
t1 = time.time()
print('SIREN:', t1-t0)
psnr_amp, psnr_pha, ssim_amp, ssim_pha, reg_amp_best, reg_pha_best = evaluate(rec_amp.cpu(),rec_phase.cpu(), amplitude.cpu(), phase.cpu(), amp_name, phase_name, image_size, probe_size, registration=False, log=True)
print('psnr_amp:', psnr_amp)
print('psnr_pha:', psnr_pha)
print('ssim_amp:', ssim_amp)
print('ssim_pha:', ssim_pha)
fig, axes = plt.subplots(1,2, figsize=(18,6))
axes[0].imshow(reg_amp_best, interpolation='nearest')
axes[1].imshow(reg_pha_best, interpolation='nearest')
plt.show()